In [1]:
# Instalação da Biblioteca
!pip install pulp -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 34.1 MB/s eta 0:00:00


In [2]:
# Importação da Biblioteca
from pulp import *

# 4º)
Um construtor de casas pré-fabricadas decidiu subcontratar quatro componentes das residências. Diversas empresas estão interessadas em participar do negócio, mas nenhuma pode aceitar mais do que um subcontrato. As cotações feitas pelas empresas para os diversos subcontratos são apresentadas na tabela a seguir. Assumindo que todas as empresas podem executar cada subcontrato igualmente bem, para que empresa cada subcontrato deve ser atribuído se o construtor de casas pré-fabricadas quiser minimizar as subcontratadas?

# Cotações por empresas (em R$ 1.000) para diversos subcontratos
| Componente | Empresa A | Empresa B | Empresa C | Empresa D |
|------------|-----------|-----------|-----------|-----------|
| 1          | 185       | 225       | 193       | 207       |
| 2          | 200       | 190       | 175       | 225       |
| 3          | 330       | 320       | 315       | 300       |
| 4          | 375       | 389       | 425       | 445       |

---

In [5]:
# Criação do problema e modelagem

prob = LpProblem("Minimizar_subcontratações", LpMinimize)

# Variáveis de decisão binárias
x = {
    (i, j): LpVariable(f"x_{i}_{j}", 0, 1, cat="Binary")
     for i in range(1, 5) for j in ['A', 'B', 'C', 'D']
    }

# Custos de atribuição (extraídos da tabela)
custos = {
    (1, 'A'): 185, (1, 'B'): 225, (1, 'C'): 193, (1, 'D'): 207,
    (2, 'A'): 200, (2, 'B'): 190, (2, 'C'): 175, (2, 'D'): 225,
    (3, 'A'): 330, (3, 'B'): 320, (3, 'C'): 315, (3, 'D'): 300,
    (4, 'A'): 375, (4, 'B'): 389, (4, 'C'): 425, (4, 'D'): 445
}

# Função objetivo
prob += lpSum(custos[(i, j)] * x[(i, j)] for i in range(1, 5) for j in ['A', 'B', 'C', 'D'])

# Restrições: Cada componente é atribuído a uma empresa
for i in range(1, 5):
    prob += lpSum(x[(i, j)] for j in ['A', 'B', 'C', 'D']) == 1

# Restrições: Cada empresa é atribuída no máximo a um componente
for j in ['A', 'B', 'C', 'D']:
    prob += lpSum(x[(i, j)] for i in range(1, 5)) <= 1

# Resolver o problema
prob.solve()

# Visualizar a melhor solução
print("Solução ótima  e custos individuais:")
total = 0
somatorio = []
for i in range(1, 5):
    for j in ['A', 'B', 'C', 'D']:
        if x[(i, j)].varValue == 1:
            custo = custos[(i, j)]
            total += custo
            somatorio.append(custo)
            print(f"Componente {i} -> Empresa {j} | Custo: R$ {custo} mil")

# Mostrar o somatório do custo total
somatorio_str = " + ".join(f"{c}" for c in somatorio)
print(f"\nCusto total mínimo: {somatorio_str} = R$ {total:.2f} mil")

Solução ótima  e custos individuais:
Componente 1 -> Empresa A | Custo: R$ 185 mil
Componente 2 -> Empresa C | Custo: R$ 175 mil
Componente 3 -> Empresa D | Custo: R$ 300 mil
Componente 4 -> Empresa B | Custo: R$ 389 mil

Custo total mínimo: 185 + 175 + 300 + 389 = R$ 1049.00 mil


Os resultados obtidos apresentam uma solução ótima com custo total mínimo de R$ 1.049.000,00. Cada componente foi atribuído à empresa correspondente de maneira a minimizar os custos, com a seguinte distribuição: o Componente 1 foi alocado à Empresa A, o Componente 2 à Empresa C, o Componente 3 à Empresa D, e o Componente 4 à Empresa B. Essa alocação reflete o equilíbrio entre os custos de produção e os critérios do problema, garantindo que os recursos sejam utilizados de forma eficiente.

# 5º)
Considere o plano logístico para quatro instalações atenderem sete cidades do Nordeste, onde são fornecidos os custos de transporte, as quantidades transportadas e o relatório da análise. Deve-se salientar que pelo menos **80% dos destinos devem ter suas respectivas demandas atendidas**.

---

## Dados do Problema

### Custo Unitário de Transporte por Distribuidor (em R$)
| **Instalação / Cidade** | **Barcelona** | **João Pessoa** | **Natal** | **Fortaleza** | **Quixadá** | **Parelhas** | **Mossoró** | **Capacidade** | **Custo Produção** |
|--------------------------|---------------|------------------|-----------|---------------|-------------|--------------|-------------|----------------|--------------------|
| **Caicó**               | 2,50          | 2,75             | 1,75      | 2,00          | 2,10        | 1,80         | 1,65        | 18.000         | 35,50             |
| **Currais Novos**        | 1,85          | 1,90             | 1,50      | 1,60          | 1,00        | 1,90         | 1,85        | 15.000         | 37,50             |
| **Tabuleiro do Norte**   | 2,30          | 2,25             | 1,85      | 1,25          | 1,50        | 2,25         | 2,00        | 25.000         | 39,00             |
| **Angicos**              | 1,90          | 0,90             | 1,60      | 1,75          | 2,00        | 2,50         | 2,65        | 20.000         | 36,25             |         |

### Demandas por Cidade
| **Cidade**       | **Demanda** |
|-------------------|-------------|
| **Barcelona**    | 8.500       |
| **João Pessoa**  | 14.500      |
| **Natal**        | 13.500      |
| **Fortaleza**    | 12.600      |
| **Quixadá**      | 18.000      |
| **Parelhas**     | 15.000      |
| **Mossoró**      | 9.000       |

---


In [15]:

# Informações do problema
instalacoes = ["Caicó", "Currais Novos", "Tabuleiro do Norte", "Angicos"]
cidades = ["Barcelona", "João Pessoa", "Natal", "Fortaleza", "Quixadá", "Parelhas", "Mossoró"]

custos_transporte = {
    "Caicó": [2.50, 2.75, 1.75, 2.00, 2.10, 1.80, 1.65],
    "Currais Novos": [1.85, 1.90, 1.50, 1.60, 1.00, 1.90, 1.85],
    "Tabuleiro do Norte": [2.30, 2.25, 1.85, 1.25, 1.50, 2.25, 2.00],
    "Angicos": [1.90, 0.90, 1.60, 1.75, 2.00, 2.50, 2.65]
}

custos_producao = {"Caicó": 35.50, "Currais Novos": 37.50, "Tabuleiro do Norte": 39.00, "Angicos": 36.25}
capacidades = {"Caicó": 18000, "Currais Novos": 15000, "Tabuleiro do Norte": 25000, "Angicos": 20000}
demandas = [8500, 14500, 13500, 12600, 18000, 15000, 9000]

# Problema de otimização
problema = LpProblem("Plano_Logistico", LpMinimize)

# Variáveis de decisão
x = LpVariable.dicts("X", [(i, j) for i in instalacoes for j in cidades], lowBound=0, cat="Continuous")
y = LpVariable.dicts("Y", instalacoes, lowBound=0, cat="Continuous")

# Função objetivo: custo total (transporte + produção)
problema += (
    lpSum(custos_transporte[i][cidades.index(j)] * x[(i, j)] for i in instalacoes for j in cidades) +
    lpSum(custos_producao[i] * y[i] for i in instalacoes)
)

# Restrições de capacidade das instalações
for i in instalacoes:
    problema += lpSum(x[(i, j)] for j in cidades) <= capacidades[i], f"Capacidade_{i}"

# Restrições de demanda mínima por cidade (80% das demandas)
for j in range(len(cidades)):
    problema += lpSum(x[(i, cidades[j])] for i in instalacoes) >= 0.8 * demandas[j], f"Demanda_{cidades[j]}"

# Relacionar produção total por instalação
for i in instalacoes:
    problema += lpSum(x[(i, j)] for j in cidades) == y[i], f"Producao_{i}"

# Solução
problema.solve()

# Exibir os resultados
print("Solução Ótima")
print(f"Custo Total: R$ {value(problema.objective):,.2f}\n")

print("Quantidades transportadas:")
for i in instalacoes:
    for j in cidades:
        if x[(i, j)].varValue > 0:
            print(f"De {i} para {j}: {x[(i, j)].varValue:.2f} unidades")

print("\nProdução total por instalação:")
for i in instalacoes:
    total_producao = sum(x[(i, j)].varValue for j in cidades)
    print(f"{i}: {total_producao:.2f} unidades")


Solução Ótima
Custo Total: R$ 2,805,450.00

Quantidades transportadas:
De Caicó para Parelhas: 12000.00 unidades
De Caicó para Mossoró: 6000.00 unidades
De Currais Novos para Natal: 600.00 unidades
De Currais Novos para Quixadá: 14400.00 unidades
De Tabuleiro do Norte para Natal: 8600.00 unidades
De Tabuleiro do Norte para Fortaleza: 10080.00 unidades
De Tabuleiro do Norte para Mossoró: 1200.00 unidades
De Angicos para Barcelona: 6800.00 unidades
De Angicos para João Pessoa: 11600.00 unidades
De Angicos para Natal: 1600.00 unidades

Produção total por instalação:
Caicó: 18000.00 unidades
Currais Novos: 15000.00 unidades
Tabuleiro do Norte: 19880.00 unidades
Angicos: 20000.00 unidades


Os resultados obtidos demonstram uma solução logística eficiente que respeita todas as restrições do problema, garantindo a minimização dos custos totais de transporte e produção, estimados em R$ 2.805.450,00. Todas as demandas mínimas de 80% por cidade foram atendidas, evidenciando a adequação do modelo em distribuir os produtos de forma estratégica. As capacidades máximas das instalações também foram respeitadas, com destaque para Caicó, Currais Novos, Tabuleiro do Norte e Angicos, que operaram com produções totais dentro de seus limites, otimizando os recursos disponíveis.